In [2]:
import pandas as pd
from constants import DATA_PATH, EOS_FILE, SENTINEL_FILE

sentinel = pd.read_csv(DATA_PATH / SENTINEL_FILE)
eos = pd.read_csv(DATA_PATH / EOS_FILE)

In [3]:
target_column = 'SM1 (%)'
target_value = 50.0

df_majority = eos[eos[target_column] == target_value]

df_minority = eos[eos[target_column] != target_value]

target_fraction = 0.3

new_majority_size = int(len(df_majority) * target_fraction)

df_majority_undersampled = df_majority.sample(
    n=new_majority_size, 
    random_state=42  # 'random_state' ensures the sampling is repeatable
)

eos = pd.concat([df_majority_undersampled, df_minority])

In [4]:
from constants import X_cols_eos, X_cols_sentinel, y_col

X_sentinel = sentinel[X_cols_sentinel].values
X_eos = eos[X_cols_eos].values

y_sentinel = sentinel[y_col].values
y_eos = eos[y_col].values

In [5]:
import tensorflow as tf

2025-11-02 11:06:22.668330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
models = {
    "16, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "8, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "2, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "4, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout, 8, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
}

In [ ]:
from model_experiments import PredictionIntervalEstimation

tf.keras.backend.clear_session()

eos_results = {}

for param_string, model in models.items():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_eos, y_eos, satellite="EOS-04")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    eos_results[param_string] = results

Epochs:   1%|▏         | 14/1000 [00:03<04:07,  3.98epoch/s, loss=0.4784, val_loss=0.4928]


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, 1: {
    "val": {
        "PICP": 0.9634703196347032,
        "MPIW": 47.19537353515625
    },
    "test": {
        "PICP": 0.9634703196347032,
        "MPIW": 47.139892578125
    }
}


Epochs:   2%|▏         | 16/1000 [00:03<04:01,  4.07epoch/s, loss=0.4795, val_loss=0.4916]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8, 1: {
    "val": {
        "PICP": 0.954337899543379,
        "MPIW": 46.81431198120117
    },
    "test": {
        "PICP": 0.9497716894977168,
        "MPIW": 46.75381851196289
    }
}


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.9497716894977168,
        "MPIW": 51.07823944091797
    },
    "test": {
        "PICP": 0.958904109589041,
        "MPIW": 51.66743087768555
    }
}


Epochs:   2%|▏         | 15/1000 [00:03<04:15,  3.86epoch/s, loss=0.4825, val_loss=0.4935]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.954337899543379,
        "MPIW": 50.81865310668945
    },
    "test": {
        "PICP": 0.9680365296803652,
        "MPIW": 51.275577545166016
    }
}


In [8]:
from constants import OUTPUT_PATH
import json

with open(OUTPUT_PATH / "pi_estimation" / "EOS-04_metrics.json", "w") as f:
    json.dump(eos_results, f, indent=4)

In [8]:
from model_experiments import PredictionIntervalEstimation
sentinel_results = {}

for param_string, model in models.items():
    tf.keras.backend.clear_session()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_sentinel, y_sentinel, satellite="Sentinel-1")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    sentinel_results[param_string] = results

Epochs:   4%|▎         | 36/1000 [00:07<03:24,  4.71epoch/s, loss=0.5720, val_loss=0.5358]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, 1: {
    "val": {
        "PICP": 0.9725274725274725,
        "MPIW": 45.59107208251953
    },
    "test": {
        "PICP": 0.9725274725274725,
        "MPIW": 45.51350402832031
    }
}


Epochs:   6%|▌         | 55/1000 [00:11<03:12,  4.92epoch/s, loss=0.5715, val_loss=0.5373]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8, 1: {
    "val": {
        "PICP": 0.978021978021978,
        "MPIW": 45.70448684692383
    },
    "test": {
        "PICP": 0.9725274725274725,
        "MPIW": 45.619407653808594
    }
}


Epochs:   6%|▌         | 58/1000 [00:11<03:12,  4.89epoch/s, loss=0.5728, val_loss=0.5368]


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4, 1: {
    "val": {
        "PICP": 0.9505494505494505,
        "MPIW": 45.704402923583984
    },
    "test": {
        "PICP": 0.9560439560439561,
        "MPIW": 45.632266998291016
    }
}


Epochs:   4%|▍         | 38/1000 [00:07<03:00,  5.34epoch/s, loss=0.5829, val_loss=0.5463]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.967032967032967,
        "MPIW": 53.326881408691406
    },
    "test": {
        "PICP": 0.945054945054945,
        "MPIW": 54.24917221069336
    }
}


Epochs:   4%|▍         | 43/1000 [00:09<03:21,  4.74epoch/s, loss=0.5734, val_loss=0.5374]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.9615384615384616,
        "MPIW": 53.28468322753906
    },
    "test": {
        "PICP": 0.945054945054945,
        "MPIW": 54.22084426879883
    }
}


In [11]:
from constants import OUTPUT_PATH
import json

with open(OUTPUT_PATH / "pi_estimation" / "Sentinel-1_metrics.json", "w") as f:
    json.dump(sentinel_results, f, indent=4)